In [ ]:
# 프로그램 시작시 해 줘야 하는 부분
# 터미널은 conda activate venv 하고 시작
import os
os.environ["CUDA_DEVICES_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="8"


# 크롤러를 만들기 전 필요한 도구 임포트
import requests
import pandas as pd
from bs4 import BeautifulSoup
import datetime
import os
import urllib3
import asyncio
import requests
import aiohttp
import traceback

# 시작 날짜와 끝 날짜, 현재 하고 있는 날짜
# start_date = '20220808'
# end_date = '20220813'
# doing_date = start_date
news_code = 101 # 뉴스 코드
start = 1 # url 크롤링 시작 페이지(default = 1)
end = 500 # url 크롤링 끝 페이지
stop_count = 150
stop_page = 25

# 뉴스 코드 리스트
idx2word = {'101' : '경제', '102' : '사회', '103' : '생활/문화', '105' : 'IT/과학'}

# 페이지 수, 카테고리, 날짜를 입력값으로 받기
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.90 Safari/537.36'}

# url 리스트받아오기
async def make_urllist(page_start, page_end, code, date, news_data, doing_date, session, day_count):
  print('뉴스 기사 하루 시작  '+ doing_date)
  page_count = 0
  url_list = []
  first_url=''
  for i in range(page_start, page_end + 1):
    url = 'https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1='+str(code)+'&date='+str(date)+'&page='+str(i)
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.90 Safari/537.36'}

    # ssl 인증서에 대한 오류(신뢰할 수 없는 ssl 인증서)가 생기기 때문에 verify=False 를 추가해준다.
    # 경고문이 뜨기 때문에 
    # import urllib3
    # urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
    # 을 추가해준다.
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
    news = requests.get(url, headers=headers, verify=False)

    try:
      # BeautifulSoup의 인스턴스 생성한다. 파서는 html.parser를 사용
      soup = BeautifulSoup(news.content, 'html.parser')

      # CASE 1
      news_list = soup.select('.newsflash_body .type06_headline li dl')
      # CASE 2
      news_list.extend(soup.select('.newsflash_body .type06 li dl'))
      page_count += 1
      # 마지막 페이지인 경우(없는 페이지를 url에 입력하면 맨 마지막 페이지가 중복으로 나오기 때문에 처음 url을 비교)
      if first_url == news_list[0]:
        print("breakpoint")
        break
      # 각 뉴스로부터 a 태그인 <a href ='주소'> 에서 '주소'만을 가져오기
      for line in news_list:
        url_list.append(line.a.get('href'))
      for line in news_list:
        news_data.append({
          'link': line.a.get('href')
        })
      first_url = news_list[0]
      if page_count >= stop_page :
        page_count = 0
        await asyncio.sleep(0.5)
    except:
      print("urllist error occured, wait 3 sec")
      await asyncio.sleep(3.0)
  print('make_urllist', len(news_data))

  # url에 해당하는 뉴스 데이터 받아오기
  await make_data(news_code, news_data, doing_date, session, doing_date)


# 날짜 단위로 크롤링
# 해당 날짜의 url을 다 받으면 시작하며 비동기적으로 진행
async def make_data(code, news_data, date, session, doing_date):

  save_folder = doing_date[:6]
  print("readStart!!" + date)
  text_list = []
  title_list = []
  date_list = []
  news_data.reverse() # 뉴스 기사 생성 시간 순으로 정렬
  count = 0
  for news in news_data:
    count += 1
    news_url = news['link']

    try:
      async with session.get(news_url, headers=headers) as res:
        html = await res.text()
        soup = BeautifulSoup(html, 'lxml')

        try:
          news_title = soup.select_one("#title_area").text.replace("\n","").replace("\t","")
          news_content = soup.select_one("#dic_area").text.replace("\n","").replace("\t","")
        except:
          continue

        try:
          html_date = soup.select_one("div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span")
          news_date = html_date.attrs['data-date-time']
        except AttributeError:
          news_date = soup.select_one("#content > div.end_ct > div > div.article_info > span > em")

        date_list.append(news_date)
        title_list.append(news_title)
        text_list.append(news_content)

      if count >= stop_count:
        print(str(doing_date) + " 기사 " + str(stop_count) + "개 입니다.")
        count = 0
        await asyncio.sleep(0.5)  # 부하로 인한 차단을 막기 위해 일시정지 후 재시작
    except:
      print(str(doing_date) + " 데이터 읽기 오류로 1초 휴식")
      await asyncio.sleep(1.0)
      print(str(doing_date) + "데이터 읽기 재시작")

  df = pd.DataFrame({'date': date_list, 'title': title_list,'news': text_list})
  df['code'] = idx2word[str(code)]
  date_list[:5]

  # 중복된 데이터 제거
  df.drop_duplicates(subset=['news'], inplace=True)
  print('최종 뉴스 기사의 개수: ',len(df))

  # 폴더가 없다면 년월에 해당하는 폴더 생성
  try:
    if not os.path.exists(save_folder):
      os.makedirs(save_folder)
  except:
    print(date)

  # csv 파일로 저장
  csv_path = save_folder + "/" + str(news_code) + "-" + date + ".csv"
  df.to_csv(csv_path, index=False)

  if os.path.exists(csv_path):
    print('{} File Saved!'.format(csv_path))


# 끝 날짜보다 작거나 같을 때 까지만 크롤링 진행
async def main():
  start_date = '20190607'
  end_date = '20190701'
  doing_date = start_date
  tasks = []
  day_count = 0
  count_days = 0
  async with aiohttp.ClientSession() as session:
    while(doing_date <= end_date):
      try:
        news_data = []
        # 해당하는 코드의 뉴스 url 가져오기
        task = make_urllist(start, end, news_code, doing_date, news_data, doing_date, session, day_count)
        count_days += 1
        print(doing_date)

        tasks.append(task)

        # 10일 단위로 데이터 가져오기
        if count_days >= 10 or doing_date == end_date:
          await asyncio.gather(*tasks)
          tasks = []
          count_days = 0
          await asyncio.sleep(2.0)  # 부하로 인한 차단을 막기 위해 일시정지 후 재시작

      # 예외가 발생했을 경우 (보통 네트워크 에러) 20초 기다린 후 재시작
      except RuntimeError:
        print("런타임에러")
        await asyncio.sleep(20.0)
        tasks = []
      except:
        print("error발생, 20초 휴식 후 재시작")
        error_msg = traceback.format_exc()
        print(error_msg)
        await asyncio.sleep(20.0)
        print("restart!!!!!")
        # continue


      # 날짜 +1 해주기
      doing = datetime.datetime.strptime(doing_date, '%Y%m%d')
      doing = doing + datetime.timedelta(days = 1)
      doing_date = datetime.datetime.strftime(doing, "%Y%m%d")

    # if len(tasks)>0 :
    #   await asyncio.gather(*tasks)
    print("end")
# .py 파일로 실행시키려면 밑의 줄 실행
#asyncio.run(main())
await main()